# Projet 7 du parcours Machine Learning (2/2)

## 2. Etat de l'art
    
    2.1 Import des données
    2.2 Import du modèle
    2.2 Evaluation sur le jeu de validation
    2.3 Evaluation sur le jeu de test

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras import regularizers
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd
from sklearn.model_selection import train_test_split


## 2.1 Import des données

In [3]:
cp -R '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train' '/root/'

In [4]:
cp -R '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test' '/root/'

In [11]:
# Création de deux listes comprenant tous les fichiers du train set et
import glob
train_list = glob.glob('/root/train/*/*.jpg')
test_list = glob.glob('/root/test/*/*.jpg')

In [12]:
labels = [train_list[x].split('/')[-2] for x in range(0,len(train_list))]

In [13]:
# Création du val set
train_list, valid_list = train_test_split(train_list, 
                                          train_size=5000,
                                          stratify=labels,
                                          random_state=42)

In [14]:
len(train_list)

5000

In [15]:
labels = [train_list[x].split('/')[-2] for x in range(0,len(train_list))]

In [17]:
mkdir 'valid'

In [18]:
import pathlib
valid_path = pathlib.Path('/root/valid')

In [20]:
for label in set(labels):
    os.mkdir(valid_path / label)

In [21]:
import shutil

In [23]:
valid_path = pathlib.Path('/root/valid')

In [24]:
for i in range(0, len(valid_list)):
    shutil.move(valid_list[i], valid_path/valid_list[i].split('/')[-2])

In [25]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.4 MB/s eta 0:00:0000:0100:01


In [26]:
import timm

In [27]:
timm.list_models('vit*', pretrained=True)

['vit_base_patch8_224',
 'vit_base_patch8_224_dino',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_dino',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_224_sam',
 'vit_base_patch16_384',
 'vit_base_patch16_rpn_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_224_sam',
 'vit_base_patch32_384',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_huge_patch14_224_in21k',
 'vit_large_patch16_224',
 'vit_large_patch16_224_in21k',
 'vit_large_patch16_384',
 'vit_large_patch32_224_in21k',
 'vit_large_patch32_384',
 'vit_large_r50_s32_224',
 'vit_large_r50_s32_224_in21k',
 'vit_large_r50_s32_384',
 'vit_relpos_base_patch16_224',
 'vit_relpos_base_patch16_clsgap_224',
 'vit_relpos_base_patch32_plus_rpn_256',
 'vit_relpos_medium_patch16_224',
 'vit_relpos_medium_patch16_cls_224',
 'vit_relpos_medium_patch16_rpn_224',
 'vit_relpos_small_patch16_224',
 'vi

## 2.2 Import du modèle


In [28]:
model = timm.create_model( 'vit_small_r26_s32_224_in21k',num_classes=196, pretrained=True)

In [29]:
from timm.data.transforms_factory import create_transform

transformator = create_transform(224)

In [30]:
import pathlib

In [31]:
train_datapath = pathlib.Path('/root/train')

In [32]:
test_datapath = pathlib.Path('/root/test')

In [33]:
valid_datapath = pathlib.Path('/root/valid')

In [34]:
from timm.data import create_dataset
from timm.data import create_loader

train_ds=create_dataset('stanford_cars', train_datapath, transform=transformator)

In [35]:
test_ds=create_dataset('stanford_cars', test_datapath, transform=transformator)

In [36]:
valid_ds=create_dataset('stanford_cars', valid_datapath, transform=transformator)

In [38]:
!pip install pytorch_accelerated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 540.3 kB/s eta 0:00:00a 0:00:01


## 2.2 Evaluation sur le jeu de validation


In [39]:
import timm
import timm.data
import timm.loss
import timm.optim
import timm.utils
import torch
import torchmetrics
from timm.scheduler import CosineLRScheduler
from pytorch_accelerated.callbacks import SaveBestModelCallback
from pytorch_accelerated.trainer import Trainer, DEFAULT_CALLBACKS

In [40]:
    image_size = (224, 224)
    lr = 5e-4
    smoothing = 0.1
    mixup = 0.2
    cutmix = 1.0
    batch_size = 16
    bce_target_thresh = 0.2
    num_epochs = 50
    num_classes = len(list(train_datapath.iterdir()))
    mixup_args = dict(
        mixup_alpha=mixup,
        cutmix_alpha=cutmix,
        label_smoothing=smoothing,
        num_classes=num_classes,
    )

In [41]:
optimizer = timm.optim.create_optimizer_v2(model.parameters(), opt='lookahead_adam', lr=0.0001)

In [42]:
from timm import loss
train_loss_fn = timm.loss.BinaryCrossEntropy(target_threshold=bce_target_thresh, smoothing=smoothing)
validate_loss_fn = torch.nn.CrossEntropyLoss()

In [43]:
class TimmMixupTrainer(Trainer):
    def __init__(self, eval_loss_fn, mixup_args, num_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_loss_fn = eval_loss_fn
        self.num_updates = None
        self.mixup_fn = timm.data.Mixup(**mixup_args)

        self.accuracy = torchmetrics.Accuracy(num_classes=num_classes)
        self.ema_accuracy = torchmetrics.Accuracy(num_classes=num_classes)
        self.ema_model = None

    def create_scheduler(self):
        return timm.scheduler.CosineLRScheduler(
            self.optimizer,
            t_initial=self.run_config.num_epochs,
            cycle_decay=0.5,
            lr_min=1e-4,
            t_in_epochs=True,
            warmup_t=3,
            warmup_lr_init=1e-4,
            cycle_limit=1,
        )

    def training_run_start(self):
        # Model EMA requires the model without a DDP wrapper and before sync batchnorm conversion
        self.ema_model = timm.utils.ModelEmaV2(
            self._accelerator.unwrap_model(self.model), decay=0.9
        )
        if self.run_config.is_distributed:
            self.model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(self.model)

    def train_epoch_start(self):
        super().train_epoch_start()
        self.num_updates = self.run_history.current_epoch * len(self._train_dataloader)

    def calculate_train_batch_loss(self, batch):
        xb, yb = batch
        mixup_xb, mixup_yb = self.mixup_fn(xb, yb)
        return super().calculate_train_batch_loss((mixup_xb, mixup_yb))

    def train_epoch_end(
        self,
    ):
        self.ema_model.update(self.model)
        self.ema_model.eval()

        if hasattr(self.optimizer, "sync_lookahead"):
            self.optimizer.sync_lookahead()

    def scheduler_step(self):
        self.num_updates += 1
        if self.scheduler is not None:
            self.scheduler.step_update(num_updates=self.num_updates)

    def calculate_eval_batch_loss(self, batch):
        with torch.no_grad():
            xb, yb = batch
            outputs = self.model(xb)
            val_loss = self.eval_loss_fn(outputs, yb)
            self.accuracy.update(outputs.argmax(-1), yb)

            ema_model_preds = self.ema_model.module(xb).argmax(-1)
            self.ema_accuracy.update(ema_model_preds, yb)

        return {"loss": val_loss, "model_outputs": outputs, "batch_size": xb.size(0)}

    def eval_epoch_end(self):
        super().eval_epoch_end()

        if self.scheduler is not None:
            self.scheduler.step(self.run_history.current_epoch + 1)

        self.run_history.update_metric("accuracy", self.accuracy.compute().cpu())
        self.run_history.update_metric(
            "ema_model_accuracy", self.ema_accuracy.compute().cpu()
        )
        self.accuracy.reset()
        self.ema_accuracy.reset()


In [44]:
train_loss_fn = timm.loss.BinaryCrossEntropy(
        target_threshold=bce_target_thresh, smoothing=smoothing
    )
validate_loss_fn = torch.nn.CrossEntropyLoss()

In [45]:
    trainer = TimmMixupTrainer(
        model=model,
        optimizer=optimizer,
        loss_func=train_loss_fn,
        eval_loss_fn=validate_loss_fn,
        mixup_args=mixup_args,
        num_classes=num_classes,
        callbacks=[
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="accuracy", greater_is_better=True),
        ],
    )

In [46]:
import timm.data
import timm.loss
import timm.optim
import timm.utils

In [47]:
    trainer.train(
        per_device_batch_size=batch_size,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        num_epochs=num_epochs,
        create_scheduler_fn=trainer.create_scheduler,
    )


Starting training run

Starting epoch 1


100%|██████████| 313/313 [01:19<00:00,  3.96it/s]



train_loss_epoch: 0.08320589031577111


100%|██████████| 503/503 [01:44<00:00,  4.83it/s]



ema_model_accuracy: 0.004352692514657974

eval_loss_epoch: 5.297966181379455

accuracy: 0.004103966988623142

Starting epoch 2


...


ema_model_accuracy: 0.8181818127632141

eval_loss_epoch: 1.0385906734888797

accuracy: 0.7802512049674988

Starting epoch 47


100%|██████████| 313/313 [01:08<00:00,  4.57it/s]



train_loss_epoch: 0.006135824126075022


 50%|████▉     | 249/503 [00:49<00:47,  5.31it/s]

KeyboardInterrupt: 

## 2.3 Evaluation sur le jeu de test

In [48]:
cp -R '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train' '/root/'

In [50]:
train_list = glob.glob('/root/train/*/*.jpg')


In [55]:
    trainer = TimmMixupTrainer(
        model=model,
        optimizer=optimizer,
        loss_func=train_loss_fn,
        eval_loss_fn=validate_loss_fn,
        mixup_args=mixup_args,
        num_classes=num_classes,
        callbacks=[
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="accuracy", greater_is_better=True),
        ],
    )

In [ ]:
    trainer.train(
        per_device_batch_size=batch_size,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        num_epochs=num_epochs,
        create_scheduler_fn=trainer.create_scheduler,
    )


Starting training run

Starting epoch 1



100%|██████████| 509/509 [01:54<00:00,  4.43it/s]



train_loss_epoch: 0.011609660954937826



100%|██████████| 503/503 [01:43<00:00,  4.84it/s]



ema_model_accuracy: 0.7990299463272095

eval_loss_epoch: 0.8014205245003697

accuracy: 0.809227705001831

Starting epoch 2


...


100%|██████████| 503/503 [01:44<00:00,  4.82it/s]



ema_model_accuracy: 0.8582265973091125

eval_loss_epoch: 0.8293664704463993

accuracy: 0.8319860696792603

Starting epoch 13



100%|██████████| 509/509 [01:54<00:00,  4.44it/s]



train_loss_epoch: 0.006110290744280328



 71%|███████   | 355/503 [01:08<00:28,  5.14it/s]


Source : https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055#5e2d
